[![logo](https://climate.copernicus.eu/sites/default/files/custom-uploads/branding/LogoLine_horizon_CAMS.png)](https://atmosphere.copernicus.eu)

# Policy tools

 **Run the tutorial via free cloud platforms**: [![binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/ecmwf-training/cams-act6/main?labpath=05-cams-mos/policy_tools.ipynb)
[![kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ecmwf-training/cams-act6/blob/main/05-cams-mos/policy_tools.ipynb)
[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ecmwf-training/cams-act6/blob/main/05-cams-mos/policy_tools.ipynb)

The policy tools provide two APIs to consume the policy data using a client software, such as this notebook, instead of by a person.

## APIs

### Air Control Toolbox (ACT)
https://policy.atmosphere.copernicus.eu/data/{yyyymm}/{model}/{mod}_{city}_{yyyymmdd}.json
https://policy.atmosphere.copernicus.eu/data/{yyyymm}/CHIMERE/{model}_{city}_{yyyymmdd}.json

### Source Allocation
https://policy.atmosphere.copernicus.eu/data/climato/{city}-{parameter}-{model}-{year}.json

## Questions
1. Which are the 10 cities where the dust share in PM10 is highest ?
2. On average between 2021 and 2023, which country contributed most to PM10 pollution in Prague ?
3. Which are the 10 cities where the maritime sector had the largest impact on ozone daily max between June and August 2024 ?

### Import libraries

In [65]:
import os
import sys
import json
import requests
import urllib
import urllib.request
import pandas as pd
import math
import datetime
from math import ceil, sqrt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

### Question 1
Here we want to answer the question: which are the 10 cities where the dust share in PM10 is highest ? We could answer this question by using the [Policy Tools](https://policy.atmosphere.copernicus.eu/daily_source_attribution/chemical_species.php?model=CHIMERE) website and selecting "Chemical speciation" in the "Daily Source Attribution" menu. We can hereafter select the date, the city, and the model. For this question we will use the CHIMERE model. We can use the API to send a request for each city and process the data to answer our question. We start by reading the file that contains the name of the cities for which the policy webste provides the forecasts.

In [6]:
with open('cities.txt', 'r', encoding="utf8") as file:
    cities = [line.strip() for line in file]

In [7]:
num_cities = len(cities)
num_cities

80

In [99]:
def act_chimere_url(day, city):
    city = urllib.parse.quote(city) # utf-8 encode for the url
    return 'https://policy.atmosphere.copernicus.eu/data/' + day[:6] + '/CHIMERE/ACT_' + city + '_' + day + '.json'

In [75]:
day = '20240912'
city = 'Malmö'

In [76]:
act_forecast_url = act_chimere_url(day, city)
act_forecast_url

'https://policy.atmosphere.copernicus.eu/data/202409/CHIMERE/ACT_Malm%C3%B6_20240912.json'

In [79]:
act_file_name = city + '_' + day + '.json'
act_file_name

'Malmö_20240912.json'

In [80]:
with urllib.request.urlopen(act_forecast_url) as url:
    json_data = json.load(url)
    #print(json_data)

We can store the json file

In [81]:
with open(act_file_name, 'w') as f:
    json.dump(json_data, f)

We parse the json file to extract the information we need to answer our question: we look for the DUST value of the total concentration of PM10 

In [82]:
def compute_dust_fraction(json_data, day):
    dust_fraction = 0.0
    for item in json_data.get(day):
        name = item['name'] 
        polluttant = item['poll']
        if (name == 'Total' and polluttant == 'DUST'):
            dust_fraction = sum(item['hOut'][:23])
    return dust_fraction  

In [83]:
dust_fraction = compute_dust_fraction(json_data, day)
print('Dust fraction in {:s} for {:s}: {:.2f}'.format(city, day, dust_fraction))

Dust fraction in Malmö for 20240912: 0.00


Now we put everything together to send a request for each city and extract the amount of dust within the total amount of PM10

In [102]:
def dust_pm10(cities, day):
    cities_dust_pm10 = []
    for city in cities:
        act_data_url = act_chimere_url(day, city)
        with urllib.request.urlopen(act_data_url) as url:
            json_data = json.load(url)
            dust_fraction = compute_dust_fraction(json_data, day)
            #print('City: {:s}, PM10 dust fraction: {:.2f}'.format(city, dust_fraction))
            cities_dust_pm10.append(dust_fraction)
    return cities_dust_pm10

In [103]:
cities_dust_fraction_pm10 = dust_pm10(cities, '20240912')

In [104]:
cities_dust = zip(cities, cities_dust_fraction_pm10)

We sort the result and print the 10 cities with the highest share of dust in PM10

In [105]:
sorted(cities_dust, key=lambda x: x[1], reverse=True)[:10]

[('Nicosia', 56.946),
 ('Valletta', 6.936),
 ('Málaga', 6.378),
 ('Murcia', 6.189),
 ('Vilnius', 5.566),
 ('Palermo', 4.579),
 ('Budapest', 4.572),
 ('Seville', 4.363),
 ('Catania', 4.041),
 ('Katowice', 2.677)]

### Question 2
Here the question is: on average between 2021 and 2023, which country contributed most to PM10 pollution in Prague ? We could answer this question using the [policy tools](https://policy.atmosphere.copernicus.eu/daily_source_attribution/country_contribution.php?date=2024-09-10) by selecting "Country contribution" in the "Yearly Air Pollution Analysis" menu. Afterward we can select the city and the pollutant. As for question 1, we can use the API to send several requests, e.g. for several cities, and process the data to answer more complex questions than the one we are dealing with here. (there is no json file in the web page)

In [61]:
def sa_url(city, year, polluttant):
    return 'https://policy.atmosphere.copernicus.eu/data/climato/' + city + '-' + polluttant + '-ACT-' + year + '.json'

In [62]:
sa_data_url = sa_url('Oslo', '2024', 'PM10')
sa_data_url

'https://policy.atmosphere.copernicus.eu/data/climato/Oslo-PM10-ACT-2024.json'

In [64]:
sa_file_name = sa_data_url.split('/')[-1:].pop()
sa_file_name

'Oslo-PM10-ACT-2024.json'

In [65]:
 with urllib.request.urlopen(sa_data_url) as url:
    sa_data = json.load(url)
    #print(data)

In [66]:
with open(sa_file_name, 'w') as f:
    json.dump(sa_data, f)

### Question 3
Here the question is: which are the 10 cities where the maritime sector had the largest impact on ozone daily max between June and August 2024 ? The maritime sector is labeled as Ship or SHP. In order to answer this question we use the same API as for question 1.

## References
* [Colette et al. - Air Control Toolbox (ACT_v1.0): a flexible surrogate model to explore mitigation scenarios in air quality forecasts](https://gmd.copernicus.org/articles/15/1441/2022/)